# AUTOMATIZACIÓN DE DETECCIÓN EN TEXTO 

In [1]:
!pip install google-generativeai pandas openpyxl


  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.3 MB 1.5 MB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
    --------------------------------------- 0.3/13.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.2 MB 2.1 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/13.2 MB 2

In [3]:
import pandas as pd
import google.generativeai as genai
import time

In [8]:
# CONFIGURACIÓN
API_KEY = "ACÁ VA LA API"
genai.configure(api_key=API_KEY)

model = genai.GenerativeModel("models/gemini-2.0-flash")


In [9]:
# CARGAR EXCEL CON OBSERVACIONES
df = pd.read_excel("Libro1.xlsx")

In [10]:
# RESULTADOS
resultados = []

In [13]:
!pip install tqdm


Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from tqdm import tqdm
import time

In [15]:
# PROCESAR POR BLOQUES DE 5 OBSERVACIONES
bloque = []
ids_bloque = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Procesando observaciones"):
    parte_id = row["ParteEnfermeriaID"]
    internacion_id = row["InternacionID"]
    observacion = row["Observaciones"]

    bloque.append(f"{len(bloque)+1}. {observacion}")
    ids_bloque.append((parte_id, internacion_id, observacion))

    # Cuando tengamos 5 observaciones, mandamos
    if len(bloque) == 5 or index == len(df) - 1:
        prompt = f"""
Vas a analizar observaciones clínicas escritas en texto libre.

Tarea: Detectar si hay drogas o monodrogas, aunque estén mal escritas o abreviadas.  
Devolvé solo las observaciones que tengan drogas, con este formato:

[Índice]: Medicamento(s)

Observaciones:
{chr(10).join(bloque)}
        """

        response = model.generate_content(prompt)
        output = response.text.strip()

        # Procesar resultado y vincular con IDs
        for linea in output.split("\n"):
            if ":" in linea:
                idx_str, drogas = linea.split(":", 1)
                try:
                    idx = int(idx_str.strip())
                    parte_id, internacion_id, obs = ids_bloque[idx - 1]
                    resultados.append({
                        "ParteEnfermeriaID": parte_id,
                        "InternacionID": internacion_id,
                        "Observacion": obs,
                        "MedicamentosDetectados": drogas.strip()
                    })
                except:
                    continue  # si hay algo raro en la línea, lo salteamos

        # Limpiar para el próximo bloque
        bloque = []
        ids_bloque = []
        time.sleep(5)  # delay para evitar límite de 15 requests por minuto


# GUARDAR EN EXCEL
df_resultado = pd.DataFrame(resultados)
df_resultado.to_excel("drogas_detectadas.xlsx", index=False)
print("✅ Listo. Archivo creado: drogas_detectadas.xlsx")


Procesando observaciones: 100%|██████████| 2636/2636 [52:05<00:00,  1.19s/it]  


✅ Listo. Archivo creado: drogas_detectadas.xlsx
